<a href="https://colab.research.google.com/github/jai2shan/TSAI-EVA40-Assignments/blob/master/Session%2010/Assignment_10_CutOut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install asgnmt9-0.0.6-py3-none-any.whl


In [0]:
import torch
import numpy as np


class Cutout:
    """Randomly mask out one or more patches from an image.
    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, prob, n_holes, length):
        self.n_holes = n_holes
        self.length = length
        self.prob = prob

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        if np.random.random()>self.prob:
          return img
        else:
          mask = np.ones((h, w), np.float32)

          for n in range(self.n_holes):
              y = np.random.randint(h)
              x = np.random.randint(w)

              y1 = np.clip(y - self.length // 2, 0, h)
              y2 = np.clip(y + self.length // 2, 0, h)
              x1 = np.clip(x - self.length // 2, 0, w)
              x2 = np.clip(x + self.length // 2, 0, w)

              mask[y1: y2, x1: x2] = 0.

          mask = torch.from_numpy(mask)
          mask = mask.expand_as(img)
          img = img * mask

          return img

## Updates in this trial

> Changing the learning rate to 0.1

In [3]:
from torchvision import transforms
from asgnmt9.DataLoader import cifar_data_loader,View_images

transform_params = dict()
transform_params['train'] = transforms.Compose([
                                       transforms.RandomRotation(10),
                                       transforms.RandomHorizontalFlip(),  
                                       transforms.RandomAffine(0,shear=10,scale=(0.8,1.2)),  
                                       transforms.ColorJitter(brightness=0.2,contrast=0.2,saturation=0.2),
                                       transforms.ToTensor(),
                                       Cutout(n_holes=1, length=16,prob = 0.3),
                                       transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

transform_params['test'] = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

trainloader,testloader = cifar_data_loader(transform_params,BatchSize=250)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import torch

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

from asgnmt9.resnet import ResNet18
net = ResNet18().to(device)
summary(net, input_size=(3,32,32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1,

In [0]:
from asgnmt9.Training_Testing import TrainTest
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR

tt = TrainTest()
test_acc = 0
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9,weight_decay = 0.00005)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

for epoch in range(50):
    print("EPOCH:", epoch)
    tt.train_(net, device, trainloader, optimizer, criterion, epoch,L1 = False)
    scheduler.step()
    acc = tt.test_(net, device, testloader)  
    
   
print('Finished Training')

  0%|          | 0/200 [00:00<?, ?it/s]

EPOCH: 0


Loss=1.724996566772461 Batch_id=199 Accuracy=28.11: 100%|██████████| 200/200 [01:10<00:00,  2.84it/s]
  0%|          | 0/200 [00:00<?, ?it/s]


Test set: Average loss: -1.8843, Accuracy: 3997/10000 (39.97%)

EPOCH: 1


Loss=1.5533655881881714 Batch_id=199 Accuracy=42.00: 100%|██████████| 200/200 [01:10<00:00,  2.83it/s]
  0%|          | 0/200 [00:00<?, ?it/s]

...

Loss=0.879986047744751 Batch_id=199 Accuracy=65.35: 100%|██████████| 200/200 [01:11<00:00,  2.82it/s]
  0%|          | 0/200 [00:00<?, ?it/s]


Test set: Average loss: -5.0000, Accuracy: 7121/10000 (71.21%)

EPOCH: 6


Loss=0.8834431767463684 Batch_id=199 Accuracy=67.85: 100%|██████████| 200/200 [01:11<00:00,  2.82it/s]
  0%|          | 0/200 [00:00<?, ?it/s]


Test set: Average loss: -5.6683, Accuracy: 7306/10000 (73.06%)

EPOCH: 7


Loss=0.8720763325691223 Batch_id=175 Accuracy=70.97:  88%|████████▊ | 176/200 [01:02<00:08,  2.85it/s]

In [0]:
from asgnmt9.ModelPerformance import *

PlotTrainingGraphs(tt)

In [0]:
Misclassification(testloader,net,classes,device)

In [0]:
ViewModelPerformance(testloader,net,classes,device)

In [0]:
from asgnmt9.gradcam import GradCamDisplay
import PIL
img_name =['airplane.png','automobile.png','bird.png','cat.png','deer.png']

pil_image = []
for i,img in enumerate(img_name):
   pil_image.append(PIL.Image.open(img))


In [0]:
from asgnmt9.gradcam import *
def GradCamDisplay(model,pil_image,classes,device):
    normed_torch_img = []
    torch_img_list = []

    for i in pil_image:
      torch_img = transforms.Compose([
          transforms.Resize((32, 32)),
          transforms.ToTensor()])(i).to(device)
      torch_img_list.append(torch_img)
      normed_torch_img .append(transforms.Normalize([0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261])(torch_img)[None])

    def imshow(img,c = "" ):
        #img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        fig = plt.figure(figsize=(10,10))
        plt.imshow(np.transpose(npimg, (1, 2, 0)),interpolation='none')
        plt.title(c)

    for i,k in enumerate(normed_torch_img):
      images1 = [torch_img_list[i].cpu()]
      images2 =  [torch_img_list[i].cpu()]
      b = copy.deepcopy(model.to(device))
      output = model(normed_torch_img[i])
      _, predicted = torch.max(output.data, 1)
      #print(classes[int(predicted)])
      layers =  [b.layer1,b.layer2,b.layer3,b.layer4]
      
      for j in layers:
        g = GradCAM(b,j)
        mask, _= g(normed_torch_img[i])
        heatmap, result = visualize_cam(mask,torch_img_list[i] )
        images1.extend([heatmap])
        images2.extend([result])

      grid_image = make_grid(images1+images2,nrow=5)
      imshow(grid_image,c = classes[int(predicted)])


In [0]:
import copy
GradCamDisplay(net,pil_image,classes,device)